In [1]:
from numpy import *
import re
import stim
import os
import time

In [2]:
def generate_circuit(folder, rounds, errors=False, errorfile=''):
    # Assumes 0XX, 1YY, 2ZZ sequence
    # Assumes rounds % 6 = 0
    
    if errorfile != '':
        errors = True
    
    # Reading edges
    edges = []
    edge_colors = []  # 0 for green, 1 for blue, 2 for red
    for i_c in range(3):
        color = {0: 'green', 1: 'blue', 2: 'red'}[i_c]
        txt_file = open(folder + f"/" + color + f"_adj_mat.txt", 'r')
        for line in txt_file:
            edge = [int(i) for i in re.split(' ', line.strip()) if i != '']
            edges.append(edge)
            edge_colors.append(i_c)
    num_edges = len(edges)
    num_qubits = amax(concatenate((edges))) + 1

    # Reading one noncontractible loop (for two logical operators on it)
    txt_file = open(folder + f"/nontriv_loops.txt", 'r')
    for line in txt_file:
        non_triv_loop = [] # in term of its edges
        loop_verts = [int(i) for i in re.split(' ', line.strip()) if i != '']
        for i_vert in range(len(loop_verts)):
            edge_i = list(sort([loop_verts[i_vert], loop_verts[i_vert + 1 - len(loop_verts)]]))
            non_triv_loop.append(edges.index(edge_i))
        break
    
    # Determining two logical operators (Z or X Paulis on a supoort)
    logical_op_0 = [] # qubits in the support, Z Paulis
    logical_op_1 = [] # qubits in the support, X Paulis
    for edge in non_triv_loop:
        if edge_colors[edge] == 0:
            logical_op_0.append(edges[edge][0])
            logical_op_0.append(edges[edge][1])
        if edge_colors[edge] == 2:
            logical_op_1.append(edges[edge][0])
            logical_op_1.append(edges[edge][1])
    assert(len(set(logical_op_0).intersection(set(logical_op_0))) % 2 == 0) # two observables commute
    
    # Determining updates for the logical operator (edges)
    updates = [[] for _ in range(3)]
    for edge in non_triv_loop:
        updates[edge_colors[edge]].append(edge)
        
    # Generating Stim circuit
    circuit = stim.Circuit()
    
    # Initialization of the observables by measuring them
    observable_targets = []
    for vert in logical_op_0:
        observable_targets.append(stim.target_z(vert))
        observable_targets.append(stim.target_combiner())
    circuit.append('MPP', observable_targets[:-1])
    circuit.append('OBSERVABLE_INCLUDE', stim.target_rec(-1), 0)

    observable_targets = []
    for vert in logical_op_1:
        observable_targets.append(stim.target_x(vert))
        observable_targets.append(stim.target_combiner())
    circuit.append('MPP', observable_targets[:-1])
    circuit.append('OBSERVABLE_INCLUDE', stim.target_rec(-1), 1)
        
    if errors:
        txt_file = open(folder + f'/error_files/' + errorfile, 'r')
    
    for rr in range(rounds):
        target = [stim.target_x, stim.target_y, stim.target_z][rr % 3]
        
        if errors:
            txt_line = txt_file.readline()
            assert(txt_line == f'step = {rr-6}\n')

            txt_line = txt_file.readline()
            assert(txt_line == '  depol:\n')

            txt_line = txt_file.readline()
            depol_errs = [int(i) for i in re.split('[ ,]+', txt_line.strip()) if i != '']
            for i_err in range(len(depol_errs) // 2):
                vert = depol_errs[2*i_err]
                err_type = depol_errs[2*i_err+1]
                circuit.append(['X_ERROR', 'Y_ERROR', 'Z_ERROR'][err_type], vert, 1)
            
            txt_line = txt_file.readline()
            assert(txt_line == '  measurement:\n')

            txt_line = txt_file.readline()
            meas_errs = [int(i) for i in re.split('[ ,]+', txt_line.strip()) if i != '']
        
        obs_recs = []
        for i_e, edge in enumerate(edges):
            if edge_colors[i_e] == rr % 3:
                p_err = 1 if errors and i_e in meas_errs else 0 #if not errors else (1 if i_e in meas_errs else 0)
                circuit.append('MPP', [target(edge[0]), stim.target_combiner(), target(edge[1])], p_err)
                obs_recs = list(array(obs_recs) - 1)
                
                if i_e in updates[rr % 3]:
                    obs_recs.append(-1)

        circuit.append('OBSERVABLE_INCLUDE', [stim.target_rec(rec) for rec in obs_recs], 0)
        circuit.append('OBSERVABLE_INCLUDE', [stim.target_rec(rec) for rec in obs_recs], 1)
        
        circuit.append("TICK", [])
    
    observable_targets = []
    for vert in logical_op_0:
        observable_targets.append(stim.target_z(vert))
        observable_targets.append(stim.target_combiner())
    circuit.append('MPP', observable_targets[:-1])
    circuit.append('OBSERVABLE_INCLUDE', stim.target_rec(-1), 0)

    observable_targets = []
    for vert in logical_op_1:
        observable_targets.append(stim.target_x(vert))
        observable_targets.append(stim.target_combiner())
    circuit.append('MPP', observable_targets[:-1])
    circuit.append('OBSERVABLE_INCLUDE', stim.target_rec(-1), 1)
    
    if errors:
        return circuit, txt_file.readlines()[-1]
    
    return circuit

In [3]:
folder = f'../Lattices/Octagonal/distance-maximizing/H64'
dd = 4
t0 = time.time()
for errorfile in os.listdir(folder + '/error_files'):
    if errorfile[0] == '.':
        continue
    circuit, logical_flips = generate_circuit(folder, 3*dd+18, errorfile=errorfile)
    
    stim_results = average(circuit.compile_detector_sampler().sample(shots=20, append_observables=True), 0)
    if stim_results[0] == 0 and stim_results[1] == 0:
        assert(logical_flips == 'False False')
    elif stim_results[0] == 0 and stim_results[1] == 1:
        assert(logical_flips == 'False True')
    elif stim_results[0] == 1 and stim_results[1] == 0:
        assert(logical_flips == 'True False')
    elif stim_results[0] == 1 and stim_results[1] == 1:
        assert(logical_flips == 'True True')
    else:
        print('issue in stim results: ', stim_results)
        assert(False)
print('All matched')
print(time.time() - t0)

All matched
26.21858787536621


In [4]:
print(circuit)

MPP Z12*Z28*Z2*Z6
OBSERVABLE_INCLUDE(0) rec[-1]
MPP X30*X54*X4*X12*X0*X1*X6*X14
OBSERVABLE_INCLUDE(1) rec[-1]
MPP(0) X0*X3 X1*X5 X2*X6 X4*X10 X7*X19 X8*X20 X9*X17 X11*X27 X12*X28 X13*X25 X14*X37 X15*X31 X16*X32 X18*X34 X21*X42 X22*X50 X23*X44 X24*X45 X26*X47 X29*X55 X30*X56 X33*X51 X35*X53 X36*X58 X38*X46 X39*X59 X40*X48 X41*X57 X43*X60 X49*X62 X52*X63 X54*X61
OBSERVABLE_INCLUDE(0) rec[-30] rec[-24]
OBSERVABLE_INCLUDE(1) rec[-30] rec[-24]
TICK
MPP(0) Y0*Y2 Y1*Y4 Y3*Y9 Y5*Y13 Y6*Y18 Y7*Y15 Y8*Y16 Y10*Y26 Y11*Y23 Y12*Y24 Y14*Y30 Y17*Y34 Y19*Y40 Y20*Y41 Y21*Y36 Y22*Y43 Y25*Y47 Y27*Y53 Y28*Y54 Y29*Y49 Y31*Y59 Y32*Y51 Y33*Y57 Y35*Y52 Y37*Y58 Y38*Y45 Y39*Y48 Y42*Y56 Y44*Y63 Y46*Y61 Y50*Y62 Y55*Y60
OBSERVABLE_INCLUDE(0) rec[-32] rec[-31] rec[-22] rec[-14]
OBSERVABLE_INCLUDE(1) rec[-32] rec[-31] rec[-22] rec[-14]
TICK
MPP(0) Z0*Z1 Z2*Z8 Z3*Z7 Z4*Z12 Z5*Z11 Z6*Z14 Z9*Z21 Z10*Z22 Z13*Z29 Z15*Z38 Z16*Z24 Z17*Z33 Z18*Z39 Z19*Z27 Z20*Z35 Z23*Z51 Z25*Z46 Z26*Z52 Z28*Z48 Z30*Z54 Z31*Z55 Z32*Z49 Z34*Z